# PyNeuro
## Модель машинного обучения  - НЕЙРОННАЯ СЕТЬ

In [1]:
from copy import deepcopy
import numpy as np
from numpy import random as rd

In [2]:
class NeuralNetwork(object):
    
    """
    Класс Нейронная сеть
    Обучение сети проходит методом обратного распространения ошибки
    layers - объект типа list: например, layers = [4, 20, 10, 5, 1]
             4 нейрона на слое входных данных (input layers);
             3 скрытых слоя сети NN (hidden layers), содержащих 20, 10 и 5 нейронов, соотвественно;
             1 нейрон в выходном слое (output layers).
    """
    
    def __init__(self, layers, weights=None):
        self.layers = layers
        self.weights = weights
        self.activations_value = None
        self.summary_arg = None
        self.n_layers = len(layers)
    
    def __repr__(self):
        return f"NeuralNetwork:\n - input layer: {self.layers[0]} neurons;\n - output layer: {self.layers[-1]} neurons;\n - hidden layers: {len(self.layers[1:-1])}"
    
    def function_activation(self, arg):
        """
        Ф-ция активации - бинарная сигмоидальная функция с областью значений (0, 1)
        """
        fx = 1 / (1 + np.exp(-arg))
        if fx == 1:
            return 1 - 1e-9
        elif fx == 0:
            return 0 + 1e-9
        else:
            return fx
    
    def derivative_activation(self, x):
        return (1 / (1 + np.exp(-x))) * (1 - (1 / (1 + np.exp(-x))))
    
    def create(self):
        """
        Ф-ция инициилизации нейронной сети. С помощью генератора списка создает матрицы 
        весов связей weights между нейронами каждого слоя со случайными числами от -1 до 1.
        !ПРИМЕЧАНИЕ!: начальная матрица весов W[0] = 0.
        """
        self.weights = [np.mat(rd.uniform(-1, 1, (self.layers[i], 1+self.layers[i-1])))
                     if i!=0 else 0 for i in range(self.n_layers)]
        
    def run(self, input_data):
        """
        Ф-ция вычисления матрицы выходных сигналов сети. 
        input_data - входные данные (может быть несколько сразу)
        !ПРИМЕЧАНИЕ!: Значения активационной функции для входного слоя равны самим значениям
        """
        count_sample = len(input_data)
        self.activations_value = [deepcopy(input_data)]
        self.summary_arg = [0]   
        fa = np.vectorize(self.function_activation)
        
        for i in range(1, self.n_layers):
            self.activations_value[i-1] = np.c_[np.ones(count_sample), self.activations_value[i-1]]
            self.summary_arg.append(self.activations_value[i-1] * self.weights[i].T)
            self.activations_value.append(fa(self.summary_arg[i]))
            
        out_signal = self.activations_value[-1]
        return out_signal
            
    def backpropagation(self, X, Y_expect, lambd):
        """
        Метод обратного распространения ошибки
        X - входные значения
        Y_expect - ожидаемые выходные значения сети 
        """
        count_layers = self.n_layers
        count_sample = len(X)
        delta_W = [0] * count_layers
        Y_expect = np.matrix(Y_expect)
        Y = deepcopy(self.run(X))
        W = deepcopy(self.weights)
        Z = deepcopy(self.summary_arg)
        A = deepcopy(self.activations_value)
        foo_der = np.vectorize(self.derivative_activation)
        
        for n in range(0, count_sample):
            delta = [0] * (count_layers + 1)
            delta[-1] = (Y[n] - Y_expect[n]).T
            for i in range(count_layers - 1, 0, -1):
                if i > 1:
                    z = Z[i-1][n]
                    z = np.c_[[[1]], z]
                    delta[i] = np.multiply(W[i].T * delta[i + 1], foo_der(z).T)
                    delta[i] = delta[i][1:]
                delta_W[i] += delta[i + 1] * A[i - 1][n]

        for i in range(1, len(delta_W)):
            delta_W[i] = delta_W[i] / count_layers
            delta_W[i][:, 1:] = delta_W[i][:, 1:] + W[i][:, 1:] * (lambd / count_layers)
            
        return delta_W
    
    def get_error(self, X, Y, lambd):
        """
        Ф-ция вычисления ошибки выходного сигнала нейронной сети с регуляризацией
        lambd - коэффициент регуляризации
        X - входные данные
        Y - выходные значения 
        """
        count_sample = len(X)
        Y = np.matrix(deepcopy(Y))
        W = deepcopy(self.weights)
        H = self.run(X)
        
        cost = (-1 * Y.T * np.log(H) - (1 - Y.T) * np.log(1 - H)).sum(axis=0).sum(axis=1)
        regul = 0
        
        for i in range(1, len(W)):
            W[i] = np.delete(W[i], 0, axis=1)
            W[i] = np.power(W[i], 2)
            regul += W[i].sum(axis=0).sum(axis=1)
            
        return cost / count_sample + (lambd / (2 * count_sample)) * regul

    def training(self, xTrain, yTrain, xTest, yTest, alpha=0.2, lambd=0.3, maxiter=1000):
        """
        Ф-ция обучения нейронной сети - подбирает матрицу весов связей на основе обучающей выборки
        alpha - отвечает за скорость обучения (насколько сильно изменяется матрица весов в каждой итерации)
        """
        eps, mi = 1, 0
        
        while eps > 0 and mi < maxiter:
            eps = self.get_error(xTrain, yTrain, lambd)
            eps_test = self.get_error(xTest, yTest, lambd)
            delta = self.backpropagation(xTrain, yTrain, lambd)
            self.weights = [self.weights[i] - alpha * delta[i] for i in range(len(self.weights))]
            mi += 1
        
        print(f'iter: {mi}, error train: {eps[0, 0]}, error test: {eps_test[0, 0]}')
        print(f'X_test:\n{self.run(xTest)}')

## Пример обучения нейронной сети

In [3]:
layers = [4, 16, 1]

In [4]:
# Создание объекта класса Нейронная сеть
NN = NeuralNetwork(layers)
NN

NeuralNetwork:
 - input layer: 4 neurons;
 - output layer: 1 neurons;
 - hidden layers: 1

In [5]:
# Создание нейронной сети
NN.create()

In [6]:
# Обучающая выборка: если последовательность возрастающая то 1, иначе - 0
xTrain = [[1, 2.3, 4.5, 5.3],
          [1.1, 1.3, 2.4, 2.4],
          [1.9, 1.7, 1.5, 1.3],
          [2.3, 2.9, 3.3, 4.9],
          [3, 5.2, 6.1, 8.2],
          [3.31, 2.9, 2.4, 1.5],
          [4.9, 5.7, 6.1, 6.3],
          [4.85, 5.0, 7.2, 8.1],
          [5.9, 5.3, 4.2, 3.3],
          [7.7, 5.4, 4.3, 3.9],
          [6.7, 5.3, 3.2, 1.4],
          [7.1, 8.6, 9.1, 9.9],
          [8.5, 7.4, 6.3, 4.1],
          [9.8, 5.3, 3.1, 2.9]]
yTrain = [[1], [1], [0], [1], [1], [0], [1], [1], [0], [0], [0], [1], [0], [0]]

In [7]:
xTest= [[0.4, 1.9, 2.5, 3.1],
        [1.51, 2.0, 2.4, 3.8],
        [2.6, 5.1, 6.2, 7.2],
        [3.23, 4.1, 4.3, 4.9],
        [7.1, 7.6, 8.2, 9.3],
        [5.78, 5.1, 4.5, 3.55],
        [6.33, 4.8, 3.4, 2.5],
        [7.67, 6.45, 5.8, 4.31],
        [8.22, 6.32, 5.87, 3.59],
        [9.1, 8.5, 7.7, 6.1]]
yTest = [[1], [1], [1], [1], [1], [0], [0], [0], [0], [0]]

In [8]:
# Обучение созданной сети
%time NN.training(xTrain, yTrain, xTest, yTest)

iter: 1000, error train: 0.21060326991731546, error test: 0.27372583015427665
X_test:
[[0.9017941 ]
 [0.87539954]
 [0.9816439 ]
 [0.86694876]
 [0.9624906 ]
 [0.12835151]
 [0.03011409]
 [0.07604479]
 [0.03362326]
 [0.15489105]]
Wall time: 21.2 s


In [9]:
# Матрица весов после обучения нейронной сети
Weights = NN.weights
Weights

[0.0,
 matrix([[ 0.44451103,  0.35645042,  0.12623628, -0.20356467, -0.42802284],
         [ 0.57881314, -0.28981819, -0.11792614,  0.10102256,  0.26536842],
         [ 0.83010273,  0.34552385,  0.11295624, -0.22923934, -0.45648222],
         [ 0.13835472,  0.35423659,  0.13147531, -0.17823694, -0.3950082 ],
         [-0.47934611, -0.36280321, -0.12830271,  0.20948922,  0.43823053],
         [-0.49978732,  0.26038056,  0.10412215, -0.09516725, -0.2440539 ],
         [-0.28303191,  0.33284209,  0.13051118, -0.13758954, -0.33332483],
         [-0.21369951, -0.36316588, -0.13385175,  0.1882649 ,  0.41163746],
         [ 0.28102577,  0.35696148,  0.12981961, -0.19072857, -0.41199703],
         [-0.64879106, -0.35914573, -0.12308419,  0.22150999,  0.4519667 ],
         [-0.34696472, -0.36331836, -0.13123399,  0.19899805,  0.42522747],
         [ 0.77113057,  0.34804508,  0.11542513, -0.22577655, -0.45298227],
         [-0.63749752, -0.3595329 , -0.12350313,  0.22078309,  0.45119593],
      

In [10]:
# Проверка (экзамен) нейронной сети
NN.run([[1.25, 2.2, 3.15, 4.1]])

matrix([[0.93194197]])